<a href="https://colab.research.google.com/github/JNAbhishek27/KalaMitra/blob/main/KalaMitra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio faiss-cpu sentence-transformers transformers

# Imports
import gradio as gr
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
from transformers import pipeline

In [2]:
data = {
    "craft": [
        "Madhubani Painting", "Blue Pottery", "Pashmina Shawl",
        "Terracotta Jewelry", "Warli Painting", "Wooden Toys"
    ],
    "region": [
        "Bihar", "Jaipur", "Kashmir", "Odisha", "Maharashtra", "Karnataka"
    ],
    "details": [
        "Madhubani painting is a folk art from Bihar, often depicting nature, mythology, and festivals.",
        "Blue pottery from Jaipur is famous for its vibrant blue dye, made using quartz and glass.",
        "Pashmina shawls from Kashmir are handwoven from fine wool, known for warmth and luxury.",
        "Terracotta jewelry from Odisha is eco-friendly and handcrafted with clay.",
        "Warli painting from Maharashtra depicts tribal life and rituals in geometric forms.",
        "Wooden toys from Karnataka are handcrafted using natural colors and traditional techniques."
    ]
}
df = pd.DataFrame(data)

In [ ]:
embedder = SentenceTransformer("all-MiniLM-L6-v2")
corpus_embeddings = embedder.encode(df["details"].tolist(), convert_to_numpy=True)

dimension = corpus_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(corpus_embeddings)

In [ ]:
story_gen = pipeline(
    "text-generation",
    model="gpt2-medium"
)
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [5]:
def rag_query(user_input, task="story"):
    query_embedding = embedder.encode([user_input], convert_to_numpy=True)
    D, I = index.search(query_embedding, k=1)
    context = df.iloc[I[0][0]]["details"]
    region = df.iloc[I[0][0]]["region"]
    craft = df.iloc[I[0][0]]["craft"]

    if task == "story":
        prompt = f"""
        Context: {context}

        Write a story about an artisan who works in {region}
        who keeps alive the tradition of {craft}.
        Describe their pride, and the cultural value of this art form.
        """
        output = story_gen(
            prompt,
            max_length=200,
            num_return_sequences=1,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.5,
            no_repeat_ngram_size=3
        )[0]["generated_text"]

    elif task == "description":
        prompt = f"""
        Context: {context}

        Write a modern product description for {craft}, highlighting its uniqueness and cultural value.
        """
        output = summarizer(prompt, max_length=60, min_length=30, do_sample=False)[0]['summary_text']

    else:
        prompt = f"""
        Context: {context}

        Create a short, catchy Instagram caption to promote {craft} made by artisans in {region}.
        """
        output = story_gen(
            prompt,
            max_length=60,
            num_return_sequences=1,
            temperature=0.8,
            top_p=0.9,
            repetition_penalty=1.4,
            no_repeat_ngram_size=3
        )[0]["generated_text"]

    return output.strip()

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("# 🌍 AI Artisan Marketplace Assistant with RAG")
    user_input = gr.Textbox(label="Enter artisan, craft, or region (e.g., Madhubani Painting, Kashmir, Jaipur)")

    with gr.Tab("Artisan Story"):
        story_out = gr.Textbox(label="Generated Story", lines=6)
        gr.Button("Generate Story").click(lambda x: rag_query(x, "story"), inputs=user_input, outputs=story_out)

    with gr.Tab("Product Description"):
        desc_out = gr.Textbox(label="Generated Description", lines=4)
        gr.Button("Generate Description").click(lambda x: rag_query(x, "description"), inputs=user_input, outputs=desc_out)

    with gr.Tab("Social Caption"):
        cap_out = gr.Textbox(label="Generated Caption", lines=2)
        gr.Button("Generate Caption").click(lambda x: rag_query(x, "caption"), inputs=user_input, outputs=cap_out)

demo.launch(share=True)